# 웹데이터-4: 한국 포털사이트에서 노래 제목을 검색

## regex 활용해서 크롤링하기

In [2]:
import urllib
keyword='벚꽃'
f=urllib.urlopen("http://music.naver.com/search/search.nhn?query=%EB%B2%9A%EA%BD%83&target=track")
mydata=f.read();
pos=mydata.find("트랙 리스트")

In [6]:
if (pos>0): # 이 부분 무슨 말인지 모르겠다...!!!
    pos = mydata.find("_title title NPI=", pos);
    pos = mydata.find("title=", pos+20)
    pos2 = mydata.find("\"", pos+8)
    print "---", mydata[pos+7:pos2]
    
#pos 가 글자수. 그러니까, 문자는 0,1,2  문자가 없으면 - 값이 나온다. pos>0은 결국, 문자가 있으면 이라는 뜻. 
    
print len(mydata)

--- 벚꽃
222676


In [7]:
import re
p=re.compile('title=".*벚꽃.*"')
res=p.findall(mydata)

In [9]:
for item in res:
    print item

title="검색어 입력" value="벚꽃" maxlength="50" accesskey="s"
title="그녀가 보내준 벚꽃편지" ><span class="ellipsis"
title="벚꽃"
title="벚꽃 여행 2" class="_album NPI=a:album,r:1,i:497654"><span class="ellipsis"
title="벚꽃" ><span class="ellipsis"
title="벚꽃" class="_album NPI=a:album,r:2,i:170969"><span class="ellipsis"
title="벚꽃놀이" ><span class="ellipsis"
title="벚꽃"
title="벚꽃 여행 2" class="_album NPI=a:album,r:3,i:497654"><span class="ellipsis"
title="サクラ色 (Sakura Iro) (벚꽃 색) - Sony 사이버 샷 CM송" ><span class="ellipsis"
title="벚꽃" ><span class="ellipsis"
title="Sakura (벚꽃)" ><span class="ellipsis"
title="벚꽃 (MR 트랙)" ><span class="ellipsis"
title="벚꽃 (Feat. 제인제이, 진정민)" ><span class="ellipsis"
title="벚꽃 (꽃이 지다...)" ><span class="ellipsis"
title="벚꽃 (꽃이 지다...)" class="_album NPI=a:album,r:9,i:585308"><span class="ellipsis"
title="벚꽃 (Inst.)" ><span class="ellipsis"
title="벚꽃"
title="Sakura (벚꽃) (DenpO115 동일본 에이어 CM 송)" ><span class="ellipsis"
title="1집 Sakurasakumachi Monogatari (벚꽃 피는 거리이야기)" class="_album NPI=a:

## CSSSelector 활용해서 크롤링하기

In [11]:
#노래 제목이 있는 위치
# body > wrap > div.fix_conts > container > .container_inner > content\
# > div:nth-child(3) > div._tracklist_mytrack.tracklist_table._searchTrack\
# > table > tbody > tr:nth-child(2) > td.name > a.title

In [7]:
import lxml.html
from lxml.cssselect import CSSSelector
html=lxml.html.fromstring(mydata)
sel=CSSSelector('#content > div:nth-child(3) \
> div._tracklist_mytrack.tracklist_table._searchTrack \
> table > tbody > tr:nth-child(2) > td.name > a.title')
results=sel(html)

In [9]:
len(results)

1

In [14]:
for result in results:
    print result.text_content()

그녀가 보내준 벚꽃편지


In [17]:
import lxml.html
import requests
keyword='벚꽃'
r = requests.get("http://music.naver.com/search/search.nhn?query="+keyword+"&x=0&y=0")

_html=lxml.html.fromstring(r.text)

In [19]:
len(lxml.html.tostring(_html))

151981

In [20]:
from lxml.cssselect import CSSSelector

sel=CSSSelector('table[summary] > tbody > ._tracklist_move > .name > a.title')
nodes=sel(_html)

In [22]:
len(nodes)

15

In [24]:
for node in nodes:
    print node.text_content()

그녀가 보내준 벚꽃편지
벗꽃 사진
벚꽃놀이
벚꽃
サクラ色 (Sakura Iro) (벚꽃 색) - Sony 사이버 샷 CM송
벚꽃
Sakura (벚꽃)
벚꽃 (MR 트랙)
벚꽃 (Feat. 제인제이, 진정민)
벚꽃 (꽃이 지다...)
벚꽃 (Inst.)
Sakura (벚꽃) (DenpO115 동일본 에이어 CM 송)
벚꽃 (Inst.)
Cherry Blossom (벚꽃)
벗꽃 버스 여행


### 곡명, 아티스트, 앨범 모두 가져오기

In [27]:
from lxml.cssselect import CSSSelector

sel=CSSSelector('table[summary] > tbody > ._tracklist_move')
nodes=sel(_html)
print lxml.html.tostring(nodes[0])

<tr class="_tracklist_move {TRACK_TYPE}" style="display:none;" trackdata="{TRACK_DATA}">


						
							<td class="chk"><input type="checkbox" title="&#49440;&#53469;" class="_chkbox_item input_chk {TRACK_CHECK_NCLICKS}"> </td>
						
						
							<td class="order">{TRACK_NUM}</td>
						
							<td class="name">

								{PLAY_TOGGLE}
								{ADD_TOGGLE}


								<span class="_ico_title ico_title"><img height="18" width="23" alt="TITLE" src="http://static.naver.net/nmusic/201008/blank.gif"></span>
								<span class="_ico_19 ico19"><img height="18" width="13" src="http://static.naver.net/nmusic/201008/blank.gif"></span>


								{TRACK_SONG_NAME}

							</td>

							<td class="_artist artist {NO_ELL}">


                                   {ARTIST}

							</td>

						
                            <td class="album">
                                {ALBUM}
                            </td>
                        
                        
                        	<td class="like

In [29]:
_selName=CSSSelector('.name > a.title')
_selArtist=CSSSelector('._artist.artist')
_selAlbum=CSSSelector('.album > a')

In [31]:
_name=_selName(nodes[1])
_artist=_selArtist(nodes[1])
_album=_selAlbum(nodes[1])

In [35]:
print _name[0].text_content()
print _artist[0].text_content().strip() # .strip() : 공백제거
print _album[0].text_content()

그녀가 보내준 벚꽃편지
벚꽃
벚꽃 여행 2


In [37]:
for node in nodes:
    _name=_selName(node)
    _artist=_selArtist(node)
    _album=_selAlbum(node)
    if _name:
        print _artist[0].text_content().strip(),
        print "---",
        print _name[0].text_content(),
        print "---",
        print _album[0].text_content()

벚꽃 --- 그녀가 보내준 벚꽃편지 --- 벚꽃 여행 2
벚꽃 --- 벗꽃 사진 --- 벗꽃 여행
벚꽃 --- 벚꽃놀이 --- 벚꽃 여행 2
블랙아웃(Black Out) --- 벚꽃 --- 벚꽃
Angela Aki --- サクラ色 (Sakura Iro) (벚꽃 색) - Sony 사이버 샷 CM송 --- Today
화이트 피아노 --- 벚꽃 --- 2집 화이트 피아노가 전하는 따뜻한 뉴에이지 정규 2집
PBC 소년소녀합창단 --- Sakura (벚꽃) --- Voices Of Peace
미누(곰돌이) --- 벚꽃 (MR 트랙) --- 곰돌이 MR집
홍창우 --- 벚꽃 (Feat. 제인제이, 진정민) --- 夢
아몬드와 땅콩들(허은진, 김준희, 김세라, 임미란) --- 벚꽃 (꽃이 지다...) --- 벚꽃 (꽃이 지다...)
이소정 --- 벚꽃 (Inst.) --- 님마중 (New Ver.)
Ikimonogakari --- Sakura (벚꽃) (DenpO115 동일본 에이어 CM 송) --- 1집 Sakurasakumachi Monogatari (벚꽃 피는 거리이야기)
블랙아웃(Black Out) --- 벚꽃 (Inst.) --- 벚꽃
강신주 --- Cherry Blossom (벚꽃) --- 나의 노래
벚꽃 --- 벗꽃 버스 여행 --- 벗꽃 여행
